In [0]:
''''
Author : Rutvik Parekh
Objective: Finding divorce rates for couples with same and different political party affiliations 

'''


#########################################  Take the data from the 20160930 election month and store it in a dataframe. #################################

import pandas as pd
import pdb
import os
from google.colab import drive
from datetime import datetime
drive.mount('/content/drive')
import sys

#Read all files and merge into 1 ( All counties)

dataframes = []

counter = 0
total = len(os.listdir('/content/drive/My Drive/3Counties_2016')) 
for filename in os.listdir('/content/drive/My Drive/3Counties_2016'):
  data = pd.read_csv('/content/drive/My Drive/3Counties_2016/'+filename, sep="\t", header=None,low_memory=False)        
  data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
  data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
  data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])          
  dataframes.append(data)

print("Processed {} of {} files".format(counter,total))
print('\n')

new_dataframe = pd.DataFrame()
temp_dataframe = pd.DataFrame()

#for i,df in enumerate(dataframes) :
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Party Affiliation'].isin(['DEM', 'REP'])]
print(temp_dataframe.shape)



######################### Process the 20160930 voter registration data and filter the couples based on same and different political party affiliations ##################

# Separate the couples who have same and different party affiliations from 20160930 election data.

#print (temp_dataframe.head(5))
pd.options.display.max_columns = None
#temp=temp_dataframe.sample(n=20000)
temp2 = temp_dataframe
#temp2=temp_dataframe
#print(temp.head(5))
#print(temp['Residence Address Line 2'].head(5))
temp2['All'] = temp2['Residence Address Line 1'].str.cat(temp2[['Residence Address Line 2']], sep=' - ')
#print(temp['All'].head(5))
df=temp2.groupby('All')
data=pd.DataFrame()

df1 = pd.DataFrame()
df2 = pd.DataFrame()
couples1 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Birth_Date_Male", "Party_Affiliation_Male","Voter_ID_Female",  "Name_First_Female", "Name_Last_Female", "Birth_Date_Female", "Party_Affiliation_Female"])
couples2 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Birth_Date_Male", "Party_Affiliation_Male","Voter_ID_Female",  "Name_First_Female", "Name_Last_Female", "Birth_Date_Female", "Party_Affiliation_Female"])
counter = 0
#print(temp2.shape)

print(df)
for name,group in df:

    if counter%30000 == 0 :
      print("Processed {} of items".format(counter) )
    counter += 1

    if group.shape[0]==2 and group[group.Gender == 'F'].shape[0] == 1 and group[group.Gender == 'M'].shape[0] == 1:
        group_name=group.groupby('Name Last')
        for name1,group1 in group_name:
            if group1.shape[0]==2:
                data=data.append([group])
                birthdate1 = pd.to_datetime(group1.iloc[0]['Birth Date'])
                birthdate2 = pd.to_datetime(group1.iloc[1]['Birth Date'])
                birthdateDiff = abs(birthdate1 - birthdate2)
                fifteenYears = pd.Timedelta(15, 'Y')
                if birthdateDiff < fifteenYears:
                  if (group1.iloc[0,13] == group1.iloc[1,13]) :
                    df1 = df1.append(group)
                    couples1AllColumnsMale = pd.DataFrame()
                    couples1AllColumnsFemale = pd.DataFrame()
                    tempCouples1 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Birth_Date_Male","Party_Affiliation_Male"])
                    couples1AllColumnsMale = group1[group1.Gender == 'M']
                    couples1AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples1[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Birth_Date_Male", "Party_Affiliation_Male"]] = couples1AllColumnsMale[['Voter ID', 'Name First', 'Name Last', 'Birth Date', 'Party Affiliation']]
                    tempCouples1['Voter_ID_Female'] = couples1AllColumnsFemale.to_numpy()[0][0]
                    tempCouples1['Name_First_Female'] = couples1AllColumnsFemale.to_numpy()[0][2]
                    tempCouples1[ 'Name_Last_Female'] = couples1AllColumnsFemale.to_numpy()[0][1]
                    tempCouples1['Birth_Date_Female'] = couples1AllColumnsFemale.to_numpy()[0][11]
                    tempCouples1[ 'Party_Affiliation_Female'] = couples1AllColumnsFemale.to_numpy()[0][13]
                    couples1 = couples1.append(tempCouples1)
                  else :
                    df2 = df2.append(group)
                    couples2AllColumnsMale = pd.DataFrame()
                    couples2AllColumnsFemale = pd.DataFrame()
                    tempCouples2 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"])
                    couples2AllColumnsMale = group1[group1.Gender == 'M']
                    couples2AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples2[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Birth_Date_Male", "Party_Affiliation_Male"]] = couples2AllColumnsMale[['Voter ID', 'Name First', 'Name Last', 'Birth Date', 'Party Affiliation']]
                    tempCouples2['Voter_ID_Female'] = couples2AllColumnsFemale.to_numpy()[0][0]
                    tempCouples2['Name_First_Female'] = couples2AllColumnsFemale.to_numpy()[0][2]
                    tempCouples2[ 'Name_Last_Female'] = couples2AllColumnsFemale.to_numpy()[0][1]
                    tempCouples2['Birth_Date_Female'] = couples2AllColumnsFemale.to_numpy()[0][11]
                    tempCouples2[ 'Party_Affiliation_Female'] = couples2AllColumnsFemale.to_numpy()[0][13]
                    couples2 = couples2.append(tempCouples2)
print('Processing Complete')


###############################################
sys.exit(1)
##############################################

########################################################## Take the data from the 20170630 election month and store it in a dataframe. ###############################
drive.mount('/content/drive')


#Read all files and merge into 1 ( All counties)

dataframes_2017 = []
total_2017 = len(os.listdir('/content/drive/My Drive/3Counties_2017/')) 
for filename in os.listdir('/content/drive/My Drive/3Counties_2017'):
  data_2017 = pd.read_csv('/content/drive/My Drive/3Counties_2017/'+filename, sep="\t", header=None,low_memory=False)       
  data_2017.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
  data_2017 =  data_2017.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]
  data_2017 = data_2017.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])        
  dataframes_2017.append(data_2017)
print('\n')
temp_dataframe_2017 = pd.DataFrame()

#for i,df in enumerate(dataframes) :
temp_dataframe_2017 = pd.concat(dataframes_2017)
temp_dataframe_2017 = temp_dataframe_2017[temp_dataframe_2017['Party Affiliation'].isin(['DEM', 'REP'])]
print(temp_dataframe_2017.shape)




######################### Process the 201760630 voter registration data and filter the couples based on same and different political party affiliations. ##############################

#print (temp_dataframe.head(5))
pd.options.display.max_columns = None
#temp=temp_dataframe.sample(n=20000)
temp2_2017 = temp_dataframe_2017
#temp2=temp_dataframe
#print(temp.head(5))
#print(temp['Residence Address Line 2'].head(5))
temp2_2017['All'] = temp2_2017['Residence Address Line 1'].str.cat(temp2_2017[['Residence Address Line 2']], sep=' - ')
#print(temp['All'].head(5))
df_2017=temp2_2017.groupby('All')
data_2017=pd.DataFrame()

df1_2017 = pd.DataFrame()
df2_2017 = pd.DataFrame()
couples12017 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male","Birth_Date_Male", "Party_Affiliation_Male", "Voter_ID_Female",  "Name_First_Female", "Name_Last_Female", "Birth_Date_Female", "Party_Affiliation_Female"])
couples22017 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Birth_Date_Male", "Party_Affiliation_Male","Voter_ID_Female",  "Name_First_Female", "Name_Last_Female", "Birth_Date_Female", "Party_Affiliation_Female"])
counter = 0
print(temp2_2017.shape)
for name,group in df_2017:

    if counter%30000 == 0 :
      print("Processed {} of items".format(counter) )
    counter += 1

    if group.shape[0]==2 and group[group.Gender == 'F'].shape[0] == 1 and group[group.Gender == 'M'].shape[0] == 1:
        group_name=group.groupby('Name Last')
        for name1,group1 in group_name:
            if group1.shape[0]==2:
                data_2017=data_2017.append([group])
                birthdate1 = pd.to_datetime(group1.iloc[0]['Birth Date'])
                birthdate2 = pd.to_datetime(group1.iloc[1]['Birth Date'])
                birthdateDiff = abs(birthdate1 - birthdate2)
                #birthdateDiffInt = birthdateDiff.astype(int)
                fifteenYears = pd.Timedelta(15, 'Y')
                if birthdateDiff < fifteenYears:
                  if (group1.iloc[0,13] == group1.iloc[1,13]) :
                    df1_2017 = df1_2017.append(group)
                    couples12017AllColumnsMale = pd.DataFrame()
                    couples12017AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples12017 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Birth_Date_Male", "Party_Affiliation_Male"])
                    couples12017AllColumnsMale = group1[group1.Gender == 'M']
                    #display(group1.take([0]))
                    #display(couples1AllColumnsFemale.head())
                    couples12017AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples12017[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male","Birth_Date_Male", "Party_Affiliation_Male"]] = couples12017AllColumnsMale[['Voter ID', 'Name First', 'Name Last', 'Birth Date', 'Party Affiliation']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples12017['Voter_ID_Female'] = couples12017AllColumnsFemale.to_numpy()[0][0]
                    tempCouples12017['Name_First_Female'] = couples12017AllColumnsFemale.to_numpy()[0][2]
                    tempCouples12017[ 'Name_Last_Female'] = couples12017AllColumnsFemale.to_numpy()[0][1]
                    tempCouples12017['Birth_Date_Female'] = couples12017AllColumnsFemale.to_numpy()[0][11]
                    tempCouples12017[ 'Party_Affiliation_Female'] = couples12017AllColumnsFemale.to_numpy()[0][13]
                    #display(tempCouples1.head())
                    couples12017 = couples12017.append(tempCouples12017)
                  else :
                    df2_2017 = df2_2017.append(group)
                    couples22017AllColumnsMale = pd.DataFrame()
                    couples22017AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples22017 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male","Birth_Date_Male", "Party_Affiliation_Male"])
                    couples22017AllColumnsMale = group1[group1.Gender == 'M']
                    #display(group1.take([0]))
                    #display(couples1AllColumnsFemale.head())
                    couples22017AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples22017[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male","Birth_Date_Male", "Party_Affiliation_Male"]] = couples22017AllColumnsMale[['Voter ID', 'Name First', 'Name Last', 'Birth Date', 'Party Affiliation']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples22017['Voter_ID_Female'] = couples22017AllColumnsFemale.to_numpy()[0][0]
                    tempCouples22017['Name_First_Female'] = couples22017AllColumnsFemale.to_numpy()[0][2]
                    tempCouples22017[ 'Name_Last_Female'] = couples22017AllColumnsFemale.to_numpy()[0][1]
                    tempCouples22017['Birth_Date_Female'] = couples22017AllColumnsFemale.to_numpy()[0][11]
                    tempCouples22017[ 'Party_Affiliation_Female'] = couples22017AllColumnsFemale.to_numpy()[0][13]
                    #display(tempCouples1.head())
                    couples22017 = couples22017.append(tempCouples22017)



############################################# Check whether we are getting proper number of couples for all categories. ######################

display(couples1.shape)
display(couples2.shape)
display(couples12017.shape)
display(couples22017.shape)





########################################## Code for finding the divorce rate for same and different political party affiliation couples. ########################

import numpy as np
couples_samePPA_merged = pd.merge(couples1, couples12017, on=['Voter_ID_Male','Voter_ID_Female'], how='left', indicator='Exist')
#couples_samePPA_merged.drop(['Name_First_Male','Name_Last_Male','Name_First_Female','Name_Last_Female'], inplace=True, axis=1)
display(couples_samePPA_merged.head())
couples_samePPA_merged[['Exist']] = np.where(couples_samePPA_merged.Exist == 'both', True, False)
display(couples_samePPA_merged['Exist'].value_counts())

samePPA_valueCounts = couples_samePPA_merged['Exist'].value_counts().tolist()
sameDivorceRate = samePPA_valueCounts[1]/(samePPA_valueCounts[0]+samePPA_valueCounts[1])
couples_diffPPA_merged = pd.merge(couples2, couples22017, on=['Voter_ID_Male','Voter_ID_Female'], how='left', indicator='Exist')
#couples_diffPPA_merged.drop(['Name_First_Male','Name_Last_Male','Name_First_Female','Name_Last_Female'], inplace=True, axis=1)
couples_diffPPA_merged['Exist'] = np.where(couples_diffPPA_merged.Exist == 'both', True, False)

display(couples_diffPPA_merged['Exist'].value_counts())

diffPPA_valueCounts = couples_diffPPA_merged['Exist'].value_counts().tolist()
diffDivorceRate = diffPPA_valueCounts[1]/(diffPPA_valueCounts[0]+diffPPA_valueCounts[1])


##################################### Print the divorce rates for same and different political party affiliation couples. #######################

print("Same political party affiliation divorce rate:")
display(sameDivorceRate*100)
print("Different political party affiliation divorce rate:")
display(diffDivorceRate*100)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
Processed 0 of 1 files


(101337, 14)
Processed 0 of items


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:82: FutureWarning: M and Y units are deprecated and will be removed in a future version.


Processed 30000 of items


KeyboardInterrupt: ignored

In [0]:
##Calculate divorce rate based on political party affiliation and age group## 
##First set is for same PPA
##Second set is for different PPA


from datetime import datetime
from dateutil.relativedelta import relativedelta

def calcDivRate(df1,df2) :
  couples_samePPA_merged = pd.merge(df1, df2, on=['Voter_ID_Male','Voter_ID_Female'], how='left', indicator='Exist')
  display(couples_samePPA_merged.head())
  couples_samePPA_merged[['Exist']] = np.where(couples_samePPA_merged.Exist == 'both', True, False)
  display(couples_samePPA_merged['Exist'].value_counts())
  samePPA_valueCounts = couples_samePPA_merged['Exist'].value_counts().tolist()
  sameDivorceRate = samePPA_valueCounts[1]/(samePPA_valueCounts[0]+samePPA_valueCounts[1])

  return sameDivorceRate*100


couples = [{"1":couples1,"2":couples12017},{"1":couples2,"2":couples22017}]

couples_1_2016 = couples1
couples_1_2017 = couples12017



age_difference = [[0,4],[4,8],[8,12],[12,16]]

diff1_dataframe = {}
diff2_dataframe = {}
for x in range(0,2) :

  couples_1_2016 = couples[x]['1']
  couples_1_2017 = couples[x]['2']


  for age in age_difference :
    diff1_dataframe[age[0]] = pd.DataFrame()
    diff2_dataframe[age[0]] = pd.DataFrame()


  counter = 0
  for idx,ele in couples_1_2016.iterrows() :

    if counter%6000 == 0 :
      print("Processed {} of {} items".format(counter,couples_1_2016.shape[0]))

    date1 = datetime.strptime(ele['Birth_Date_Male'],'%m/%d/%Y')
    date2 = datetime.strptime(ele['Birth_Date_Female'],'%m/%d/%Y')
    difference_in_years = abs(relativedelta(date1, date2).years)
      
    for age in age_difference :
      if difference_in_years >= age[0] and difference_in_years < age[1] :
        diff1_dataframe[age[0]] = diff1_dataframe[age[0]].append(couples_1_2016.iloc[counter,:])
    counter += 1

  print("Processed {} of {} items".format(counter,couples_1_2016.shape[0]))

  counter = 0
  for idx,ele in couples_1_2017.iterrows() :
    if counter%6000 == 0 :
      print("Processed {} of {} items".format(counter,couples_1_2017.shape[0]))

    date1 = datetime.strptime(ele['Birth_Date_Male'],'%m/%d/%Y')
    date2 = datetime.strptime(ele['Birth_Date_Female'],'%m/%d/%Y')
    difference_in_years = abs(relativedelta(date1, date2).years)
      
    for age in age_difference :
      if difference_in_years >= age[0] and difference_in_years < age[1] :
        diff2_dataframe[age[0]] = diff2_dataframe[age[0]].append(couples_1_2017.iloc[counter,:])
    
    counter += 1

  print("Processed {} of {} items".format(counter,couples_1_2017.shape[0]))


  for age in age_difference :
    print("Divorce rate for age group between {} and {} is {}".format(age[0],age[1],calcDivRate(diff1_dataframe[age[0]],diff2_dataframe[age[0]])))






Processed 0 of 11183 items
Processed 6000 of 11183 items
Processed 11183 of 11183 items
Processed 0 of 11299 items
Processed 6000 of 11299 items
Processed 11299 of 11299 items


,Birth_Date_Female_x,Birth_Date_Male_x,Name_First_Female_x,Name_First_Male_x,Name_Last_Female_x,Name_Last_Male_x,Party_Affiliation_Female_x,Party_Affiliation_Male_x,Voter_ID_Female,Voter_ID_Male,Birth_Date_Female_y,Birth_Date_Male_y,Name_First_Female_y,Name_First_Male_y,Name_Last_Female_y,Name_Last_Male_y,Party_Affiliation_Female_y,Party_Affiliation_Male_y,Exist
0,11/02/1949,08/02/1946,JOYCE,DOUGLAS,BROWN,BROWN,DEM,DEM,107114152.0,107365689.0,11/02/1949,08/02/1946,JOYCE,DOUGLAS,BROWN,BROWN,DEM,DEM,both
1,06/01/1931,09/28/1930,FRANCES,DONALD,RENZ,RENZ,REP,REP,104491403.0,104491404.0,06/01/1931,09/28/1930,FRANCES,DONALD,RENZ,RENZ,REP,REP,both
2,02/21/1966,09/15/1965,LORA,JOHN,SHINABERRY,SHINABERRY,DEM,DEM,104495488.0,104495489.0,02/21/1966,09/15/1965,LORA,JOHN,SHINABERRY,SHINABERRY,DEM,DEM,both
3,08/20/1985,01/19/1985,BRITTANY,DAVID,RUSSELL,RUSSELL,REP,REP,114972475.0,104386118.0,08/20/1985,01/19/1985,BRITTANY,DAVID,RUSSELL,RUSSELL,REP,REP,both
4,12/09/1950,01/29/1948,CAROL,GERALD,SEARSON,SEARSON,REP,REP,118488708.0,118488293.0,12/09/1950,01/29/1948,CAROL,GERALD,SEARSON,SEARSON,REP,REP,both


True     6284
False     861
Name: Exist, dtype: int64

Divorce rate for age group between 0 and 4 is 12.05038488453464


,Birth_Date_Female_x,Birth_Date_Male_x,Name_First_Female_x,Name_First_Male_x,Name_Last_Female_x,Name_Last_Male_x,Party_Affiliation_Female_x,Party_Affiliation_Male_x,Voter_ID_Female,Voter_ID_Male,Birth_Date_Female_y,Birth_Date_Male_y,Name_First_Female_y,Name_First_Male_y,Name_Last_Female_y,Name_Last_Male_y,Party_Affiliation_Female_y,Party_Affiliation_Male_y,Exist
0,06/01/1969,03/19/1975,LISA,LAMBRECHT,EVERS,EVERS,REP,REP,104478653.0,122498064.0,06/01/1969,03/19/1975,LISA,LAMBRECHT,EVERS,EVERS,REP,REP,both
1,03/02/1940,08/23/1935,NORMA,TERENCE,MCLAUGHLIN,MCLAUGHLIN,DEM,DEM,104425968.0,104425969.0,03/02/1940,08/23/1935,NORMA,TERENCE,MCLAUGHLIN,MCLAUGHLIN,DEM,DEM,both
2,02/19/1955,12/22/1949,DORA,EDUARDO,RODRIGUEZ,RODRIGUEZ,REP,REP,104391243.0,104404726.0,02/19/1955,12/22/1949,DORA,EDUARDO,RODRIGUEZ,RODRIGUEZ,REP,REP,both
3,05/14/1964,11/28/1957,MARILYN,MARK,BELLAMY,BELLAMY,REP,REP,117645905.0,104472269.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,07/26/1976,07/30/1970,HEATHER,DEREK,SCHRAMM,SCHRAMM,REP,REP,122814872.0,123227494.0,07/26/1976,07/30/1970,HEATHER,DEREK,SCHRAMM,SCHRAMM,REP,REP,both


...

True     165
False     25
Name: Exist, dtype: int64

Divorce rate for age group between 8 and 12 is 13.157894736842104


,Birth_Date_Female_x,Birth_Date_Male_x,Name_First_Female_x,Name_First_Male_x,Name_Last_Female_x,Name_Last_Male_x,Party_Affiliation_Female_x,Party_Affiliation_Male_x,Voter_ID_Female,Voter_ID_Male,Birth_Date_Female_y,Birth_Date_Male_y,Name_First_Female_y,Name_First_Male_y,Name_Last_Female_y,Name_Last_Male_y,Party_Affiliation_Female_y,Party_Affiliation_Male_y,Exist
0,05/16/1976,10/26/1961,VIRGINIA,BRENT,GAUSTAD,GAUSTAD,DEM,REP,104472711.0,104455042.0,05/16/1976,10/26/1961,VIRGINIA,BRENT,GAUSTAD,GAUSTAD,DEM,REP,both
1,03/16/1955,07/12/1968,PATRICIA,ROBERT,GARELLI,GARELLI,DEM,REP,104381430.0,104381429.0,03/16/1955,07/12/1968,PATRICIA,ROBERT,GARELLI,GARELLI,DEM,REP,both
2,09/23/1980,04/05/1968,BROOKE,ADAM,BORKOWSKI,BORKOWSKI,REP,DEM,104377356.0,104414021.0,09/23/1980,04/05/1968,BROOKE,ADAM,BORKOWSKI,BORKOWSKI,REP,DEM,both
3,01/11/1944,03/30/1930,FLORENCE,LOUIS,CAMPOLI,CAMPOLI,REP,DEM,104378784.0,104378785.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,01/16/1985,02/14/1970,MELISSA,STEPHEN,TAYLOR,TAYLOR,DEM,REP,104379983.0,118999072.0,01/16/1985,02/14/1970,MELISSA,STEPHEN,TAYLOR,TAYLOR,DEM,REP,both


True     45
False    12
Name: Exist, dtype: int64

Divorce rate for age group between 12 and 16 is 21.052631578947366


In [0]:
######################################################## Code to extract divorce rates among Dem-Dem, Rep-Rep, Dem-Rpe and Rep-Dem ################################

demdemCouples1 = pd.DataFrame()
reprepCouples1 = pd.DataFrame()

print("Same 2016")
for _,couple1 in couples1.iterrows():
  if couple1[4] == 'DEM' and couple1[9] == 'DEM':
    demdemCouples1 = demdemCouples1.append(couple1)
  if couple1[4] == 'REP' and couple1[9] == 'REP':
    reprepCouples1 = reprepCouples1.append(couple1)

print("Same 2017")
demdemCouples12017 = pd.DataFrame()
reprepCouples12017 = pd.DataFrame()

for _,couple12017 in couples12017.iterrows():
  if couple12017[4] == 'DEM' and couple12017[9] == 'DEM':
    demdemCouples12017 = demdemCouples12017.append(couple12017)
  if couple12017[4] == 'REP' and couple12017[9] == 'REP':
    reprepCouples12017 = reprepCouples12017.append(couple12017)

print("Diff 2016")
demrepCouples2 = pd.DataFrame()
repdemCouples2 = pd.DataFrame()
for _,couple2 in couples2.iterrows():
  if couple2[4] == 'DEM' and couple2[9] == 'REP':
    demrepCouples2 = demrepCouples2.append(couple2)
  if couple2[4] == 'REP' and couple2[9] == 'DEM':
    repdemCouples2 = repdemCouples2.append(couple2)

print("Diff 2017")
demrepCouples22017 = pd.DataFrame()
repdemCouples22017 = pd.DataFrame()
for _,couple22017 in couples22017.iterrows():
  if couple22017[4] == 'DEM' and couple22017[9] == 'REP':
    demrepCouples22017 = demrepCouples22017.append(couple22017)
  if couple22017[4] == 'REP' and couple22017[9] == 'DEM':
    repdemCouples22017 = repdemCouples22017.append(couple22017)

display("Dem-Dem Couples")
display(demdemCouples1.head())
display("Rep-Rep Couples")
display(reprepCouples1.head())
display("Dem-Dem Couples 2017")
display(demdemCouples12017.head())
display("Rep-Rep Couples 2017")
display(reprepCouples12017.head())
display("Dem-Rep Couples")
display(demrepCouples2.head())
display("Dem-Rep Couples 2017")
display(demrepCouples22017.head())
display("Rep-Dem Couples")
display(repdemCouples2.head())
display("Rep-Dem Couples 2017")
display(repdemCouples22017.head())


def calculateDivorceRate(couple, couple2017):
  couples_merged = pd.merge(couple, couple2017, on=['Voter_ID_Male','Voter_ID_Female'], how='left', indicator='Exist')
  couples_merged[['Exist']] = np.where(couples_merged.Exist == 'both', True, False)
  valueCounts = couples_merged['Exist'].value_counts().tolist()
  divorceRate = valueCounts[1]/(valueCounts[0]+valueCounts[1])
  return divorceRate*100

demdemDivorceRate = calculateDivorceRate(demdemCouples1, demdemCouples12017)
reprepDivorceRate = calculateDivorceRate(reprepCouples1, reprepCouples12017)
demrepDivorceRate = calculateDivorceRate(demrepCouples2, demrepCouples22017)
repdemDivorceRate = calculateDivorceRate(repdemCouples2, repdemCouples22017)

print("Number of Dem Dem Couples: ")
print(demdemCouples1.shape)
print("Number of Dem Dem Couples in 2017: ")
print(demdemCouples12017.shape)
print("Dem-Dem Divorce Rate")
print(demdemDivorceRate)
print("\n")
print("Number of Rep Rep Couples: ")
print(reprepCouples1.shape)
print("Number of Rep Rep Couples in 2017: ")
print(reprepCouples12017.shape)
print("Rep-Rep Divorce Rate")
print(reprepDivorceRate)
print("\n")
print("Number of Dem Rep Couples: ")
print(demrepCouples2.shape)
print("Number of Dem Rep Couples in 2017: ")
print(demrepCouples22017.shape)
print("Dem-Rep Divorce Rate")
print(demrepDivorceRate)
print("\n")
print("Number of Rep Dem Couples: ")
print(repdemCouples2.shape)
print("Number of Rep Dem Couples in 2017: ")
print(repdemCouples22017.shape)
print("Rep-Dem Divorce Rate")
print(repdemDivorceRate)

In [0]:
def pcounter(current,total,step=100) :
  if current%step == 0 :
    print("Processed {} of {} items".format(current,total))

def check_divorced(dict_1,dict_2) :
  count_present = 0
  count_absent = 0
  total = len(dict_1)
  count = 0
  for k,v in dict_1.items() :
    
    count += 1
    pcounter(count,total,100000)

    if k not in dict_2.keys() :
      count_absent += 1 
    else :
      try:
        names_1 = sorted([ v[0]['Name First'].strip() ,v[1]['Name First'].strip() ])
        names_2 = sorted([ dict_2[k][0]['Name First'].strip() , dict_2[k][1]['Name First'].strip()  ])
      except Exception as e:
        pdb.set_trace()

    if names_1 == names_2 :
      count_present += 1
    else :
      count_absent +=1   
  pcounter(count,total,100000)    

  return count_present,count_absent

def identify_couples(couples) :
  hashmap = {}
  counter = 0
  for index,row in couples.iterrows() :
    counter += 1
    pcounter(counter,couples.shape[0],100000)
    val = row.to_dict()

    val['complete_address'] =  val['complete_address'].replace(' ','')
    
    key = val['complete_address'] + val["Name Last"]

    if key not in hashmap :
      hashmap[key] = []
    hashmap[key].append(val)
  pcounter(counter,couples.shape[0],100000) 
    
  same_party = {}
  diff_party = {}

  total = len(hashmap.keys())
  counter = 0
  for key,value in hashmap.items() :

    if counter%100000 == 0 :
      print("Processed {} of {} items".format(counter,total))

    counter += 1

    if len(value) > 2 or len(value) == 1:
      continue

    gen1 = value[0]['Gender'].strip()
    gen2 =  value[1]['Gender'].strip()

    if gen1 == gen2  :
      continue


    value[0]['Party Affiliation'] = value[0]['Party Affiliation'].strip()
    value[1]['Party Affiliation'] = value[1]['Party Affiliation'].strip()

    if value[0]['Party Affiliation'] == value[1]['Party Affiliation'] :
      if key not in same_party :
        same_party[key] = []
      same_party[key] = value 
    else :
      if key not in diff_party :
        diff_party[key] = []      
      diff_party[key] = value


  print("Processed {} of {} items".format(counter,total))
  
  return same_party,diff_party

In [0]:
''''
Author : Viraj Kamat
Objective: Finding divorce rates for couples with same and different political party affiliations 

'''


import pandas as pd
import pdb
import os
from google.colab import drive
from datetime import datetime
drive.mount('/content/drive')
import sys
import time
pd.options.display.max_columns = None

#Read all files and merge into 1 ( All counties)
start = time.time()
dataframes = []
counter = 0
total = len(os.listdir('/content/drive/My Drive/3Counties_2016')) 
for filename in os.listdir('/content/drive/My Drive/3Counties_2016'):
  data = pd.read_csv('/content/drive/My Drive/3Counties_2016/'+filename, sep="\t", header=None,low_memory=False)        
  data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
  data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
  data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])          
  dataframes.append(data)

temp_dataframe = pd.DataFrame()
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Party Affiliation'].isin(['DEM', 'REP'])]
temp2 = temp_dataframe
temp2['complete_address'] = temp2['Residence Address Line 1'].str.cat(temp2[['Residence Address Line 2']], sep=' - ')



dataframes = []
counter = 0
total = len(os.listdir('/content/drive/My Drive/3Counties_2017')) 
for filename in os.listdir('/content/drive/My Drive/3Counties_2017'):
  data = pd.read_csv('/content/drive/My Drive/3Counties_2017/'+filename, sep="\t", header=None,low_memory=False)        
  data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
  data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
  data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])          
  dataframes.append(data)




temp_dataframe = pd.DataFrame()
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Party Affiliation'].isin(['DEM', 'REP'])]
temp3 = temp_dataframe
temp3['complete_address'] = temp3['Residence Address Line 1'].str.cat(temp3[['Residence Address Line 2']], sep=' - ')



coupes2016_same,coupes2016_diff =  identify_couples(temp2)
coupes2017_same,coupes2017_diff =  identify_couples(temp3)


print(check_divorced(coupes2016_same,coupes2017_same))
print(check_divorced(coupes2016_diff,coupes2017_diff))
print(time.time() - start)







In [0]:
start = time.time()
dataframes = []
counter = 0
total = len(os.listdir('/content/drive/My Drive/dsf-project/20160930/')) 
for filename in os.listdir('/content/drive/My Drive/dsf-project/20160930/'):
  counter += 1
  if counter%10 == 0 :
    pcounter(counter,total,10)
  if filename.endswith('html') or filename.endswith('tmp') :
    continue
  try :
    data = pd.read_csv('/content/drive/My Drive/dsf-project/20160930/'+filename, sep="\t", header=None,low_memory=False,encoding='cp1252')        
    data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
    data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
    data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])          
    dataframes.append(data)
  except Exception as e :
    print("Failed to read {}".format(filename))
    continue
pcounter(counter,total,10)
temp_dataframe = pd.DataFrame()
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Party Affiliation'].isin(['DEM', 'REP'])]
temp2 = temp_dataframe
temp2['complete_address'] = temp2['Residence Address Line 1'].str.cat(temp2[['Residence Address Line 2']], sep=' - ')
df=temp2.groupby('complete_address')


coupes2016_same,coupes2016_diff =  identify_couples(temp2)
print(time.time() - start)

In [0]:
def pcounter(current,total,step=100) :
  if current%step == 0 :
    print("Processed {} of {} items".format(current,total))

def check_divorced(dict_1,dict_2) :
  count_present = 0
  count_absent = 0
  total = len(dict_1)
  count = 0
  for k,v in dict_1.items() :
    
    count += 1
    pcounter(count,total,100000)

    if k not in dict_2.keys() :
      count_absent += 1 
    else :
      try:
        names_1 = sorted([ v[0]['Name First'].strip() ,v[1]['Name First'].strip() ])
        names_2 = sorted([ dict_2[k][0]['Name First'].strip() , dict_2[k][1]['Name First'].strip()  ])
      except Exception as e:
        pdb.set_trace()

    if names_1 == names_2 :
      count_present += 1
    else :
      count_absent +=1   
  pcounter(count,total,100000)    

  return count_present,count_absent

def identify_couples(couples) :
  hashmap = {}
  counter = 0
  for index,row in couples.iterrows() :
    counter += 1
    pcounter(counter,couples.shape[0],100000)
    val = row.to_dict()

    val['complete_address'] =  val['complete_address'].replace(' ','')
    
    key = val['complete_address'] + val["Name Last"]

    if key not in hashmap :
      hashmap[key] = []
    hashmap[key].append(val)
  pcounter(counter,couples.shape[0],100000) 
    
  same_party = {}
  diff_party = {}

  total = len(hashmap.keys())
  counter = 0
  for key,value in hashmap.items() :

    if counter%100000 == 0 :
      print("Processed {} of {} items".format(counter,total))

    counter += 1

    if len(value) > 2 or len(value) == 1:
      continue

    gen1 = value[0]['Gender'].strip()
    gen2 =  value[1]['Gender'].strip()

    if gen1 == gen2  :
      continue


    value[0]['Party Affiliation'] = value[0]['Party Affiliation'].strip()
    value[1]['Party Affiliation'] = value[1]['Party Affiliation'].strip()

    if value[0]['Party Affiliation'] == value[1]['Party Affiliation'] :
      if key not in same_party :
        same_party[key] = []
      same_party[key] = value 
    else :
      if key not in diff_party :
        diff_party[key] = []      
      diff_party[key] = value


  print("Processed {} of {} items".format(counter,total))
  
  return same_party,diff_party



import pandas as pd
import pdb
import os
from google.colab import drive
from datetime import datetime
drive.mount('/content/drive')
import sys
import time
pd.options.display.max_columns = None

#Read all files and merge into 1 ( All counties)
start = time.time()
dataframes = []
counter = 0
total = len(os.listdir('/content/drive/My Drive/NY_2016/')) 
for filename in os.listdir('/content/drive/My Drive/NY_2016/'):
  counter += 1
  if counter%10 == 0 :
    pcounter(counter,total,10)
  if filename.endswith('html') or filename.endswith('tmp') :
    continue
  try :
    data = pd.read_csv('/content/drive/My Drive/NY_2016/'+filename, sep="\t", header=None,low_memory=False)        
    data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
    data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
    data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])          
    dataframes.append(data)
  except Exception as e :
    print("Failed to read {}".format(filename))
    continue
pcounter(counter,total,10)

temp_dataframe = pd.DataFrame()
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Party Affiliation'].isin(['DEM', 'REP'])]
temp2 = temp_dataframe
temp2['complete_address'] = temp2['Residence Address Line 1'].str.cat(temp2[['Residence Address Line 2']], sep=' - ')



dataframes = []
counter = 0
total = len(os.listdir('/content/drive/My Drive/NY_2017/')) 
for filename in os.listdir('/content/drive/My Drive/NY_2017/'):
  counter += 1
  if counter%10 == 0 :
    pcounter(counter,total,10)
  if filename.endswith('html') or filename.endswith('tmp') :
    continue
  try :
    data = pd.read_csv('/content/drive/My Drive/NY_2017/'+filename, sep="\t", header=None,low_memory=False)        
    data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
    data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
    data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])          
    dataframes.append(data)

  except Exception as e :
    print("Failed to read {}".format(filename))
    continue
pcounter(counter,total,10)



temp_dataframe = pd.DataFrame()
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Party Affiliation'].isin(['DEM', 'REP'])]
temp3 = temp_dataframe
temp3['complete_address'] = temp3['Residence Address Line 1'].str.cat(temp3[['Residence Address Line 2']], sep=' - ')



coupes2016_same,coupes2016_diff =  identify_couples(temp2)
coupes2017_same,coupes2017_diff =  identify_couples(temp3)


print(check_divorced(coupes2016_same,coupes2017_same))
print(check_divorced(coupes2016_diff,coupes2017_diff))
print(time.time() - start)



In [0]:
print(170273/(1101171+170273)*100)

print(29349/(170323+29349)*100)